In [1]:
import matplotlib.path as mpltPath
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [2]:
box_l = 200
xw = box_l/2.0
yw = box_l/2.0

border_x_list = [-xw,-xw]
border_y_list = [-yw,yw]

BinLength = box_l / 20
for bin in np.arange(1, 20):
    border_x_list.append(border_x_list[bin] + BinLength)
    border_y_list.append(yw)

border_x_list.extend([xw,xw,-xw])
border_y_list.extend([yw,-yw,-yw])

borderX = np.array(border_x_list)
borderY = np.array(border_y_list)

In [3]:
from numba import jit, prange
from numba import types as t

def is_inside_sm(polygon, point):
    length = len(polygon) - 1
    dy2 = point[1] - polygon[0][1]
    intersections = 0
    ii = 0
    jj = 1

    while ii < length:
        dy = dy2
        dy2 = point[1] - polygon[jj][1]

        # consider only lines which are not completely above/bellow/right from the point
        if dy * dy2 <= 0.0 and (point[0] >= polygon[ii][0] or point[0] >= polygon[jj][0]):

            # non-horizontal line
            if dy < 0 or dy2 < 0:
                F = dy * (polygon[jj][0] - polygon[ii][0]) / (dy - dy2) + polygon[ii][0]

                if point[0] > F:  # if line is left from the point - the ray moving towards left, will intersect it
                    intersections += 1
                elif point[0] == F:  # point on line
                    return 2

            # point on upper peak (dy2=dx2=0) or horizontal line (dy=dy2=0 and dx*dx2<=0)
            elif dy2 == 0 and (point[0] == polygon[jj][0] or (
                    dy == 0 and (point[0] - polygon[ii][0]) * (point[0] - polygon[jj][0]) <= 0)):
                return 2

            # there is another posibility: (dy=0 and dy2>0) or (dy>0 and dy2=0). It is skipped
            # deliberately to prevent break-points intersections to be counted twice.

        ii = jj
        jj += 1

    # print 'intersections =', intersections
    return intersections & 1

In [4]:
import numba
from numba import prange

def find_bound_status(x_co_ordinates, y_co_ordinates, poly_x, poly_y):
    status = np.empty(len(x_co_ordinates), dtype=bool)
    polygon = np.vstack((poly_x, poly_y)).T
    points = np.vstack((x_co_ordinates, y_co_ordinates)).T
    for i in prange(len(x_co_ordinates)):
        status[i] = is_inside_sm(polygon, points[i])
    in_bound = np.where(status)[0]
    out_bound = np.where(status == False)[0]
    return in_bound, out_bound

In [5]:
def get_random_points(points_count, spread):
    return (np.random.rand(points_count, 2) - 0.5) * spread


In [6]:
border_points = np.vstack((borderX, borderY)).T
border_path = mpltPath.Path(border_points)

In [8]:
for i in range(20):
    points = get_random_points(10000, 300)

    bound_status = border_path.contains_points(points)
    mat_in = np.where(bound_status==True)[0]
    mat_out = np.where(bound_status==False)[0]

    new_algo_in, new_algo_out = find_bound_status(points[:,0], points[:,1], borderX, borderY)

    if not np.array_equal(mat_in, new_algo_in):
        print(f"In bound particles match do not match.")
    if not np.array_equal(mat_out, new_algo_out):
        print(f"Out bound particles match do not match.")